<a href="https://colab.research.google.com/github/MaiSerry/DoctBot_grad/blob/main/pushing%26inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Push model to huggingface

In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_path = "/content/drive/MyDrive/DoctorBot_symptoms_disease"


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="maiserry/DoctorBot_symptoms_disease",
    folder_path=model_path,
    repo_type="model",
    commit_message="Push fine-tuned BERT with label encoder"
)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

label_encoder.pkl:   0%|          | 0.00/16.8k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/maiserry/DoctorBot_symptoms_disease/commit/6faaa4eed04f4c9aa748d23ac61c75adcbaee370', commit_message='Push fine-tuned BERT with label encoder', commit_description='', oid='6faaa4eed04f4c9aa748d23ac61c75adcbaee370', pr_url=None, repo_url=RepoUrl('https://huggingface.co/maiserry/DoctorBot_symptoms_disease', endpoint='https://huggingface.co', repo_type='model', repo_id='maiserry/DoctorBot_symptoms_disease'), pr_revision=None, pr_num=None)

# inference and testing

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("maiserry/DoctorBot_symptoms_disease")
tokenizer = AutoTokenizer.from_pretrained("maiserry/DoctorBot_symptoms_disease")


config.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import hf_hub_download
import joblib


path = hf_hub_download("maiserry/DoctorBot_symptoms_disease", "label_encoder.pkl")
label_encoder = joblib.load(open(path, 'rb'))


In [ ]:
def predict_disease(text):

  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
  outputs = model(**inputs)
  logits = outputs.logits
  predicted_class = torch.argmax(logits, dim=1).item()
  return label_encoder.inverse_transform([predicted_class])[0]


In [ ]:
predict_disease('I have a sick throat and fever')

'abscess of the pharynx'

In [ ]:
predict_disease('I feel like nothing matters anymore, I just want to give up.')

'smoking or tobacco addiction'